In [ ]:
# | default_exp dashboard

In [ ]:
# | export
import os
from functools import cached_property, partial
from threading import Thread
from time import sleep, time

import dash_bootstrap_components as dbc
import dash_svg as dsvg
import numpy as np
import orjson
import pandas as pd
import plotly.express as px
import plotly.io as pio
import wandb
from aifront.overlay.default import table_row
from aifront.overlay.medal import medal_icon, medal_index
from dash import Dash, Input, Output, State, dcc, html
from dash.exceptions import PreventUpdate
from dash_extensions import WebSocket
from fastcore.basics import store_attr
from fastcore.foundation import L, Self
from furl import furl

In [ ]:
# |export
class WandbFigureFactory:
    font = dict(
        size=20,
        family='Lato,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol"',
    )

    def __init__(
        self,
        entity=os.getenv("TMLEARN_WANDB_ENTITY", "trackmania-ai"),
        project=os.getenv("TMLEARN_WANDB_PROJECT", "tmaifoo"),
        samples=2000,
    ):
        store_attr()
        self.api = wandb.Api(timeout=60)

    @property
    def wandb_path(self):
        return f"{self.entity}/{self.project}"

    def collect_runs(self, state="running", group=None):
        self.api.flush()
        filters = dict(tags="collect")
        if state is not None:
            filters["state"] = state
        if group is not None:
            filters["group"] = group
        return L(self.api.runs(self.wandb_path, filters))

    def training_run(self, state="running"):
        self.api.flush()
        filters = dict(tags="training")
        if state is not None:
            filters["state"] = state
        runs = self.api.runs(self.wandb_path, filters)
        assert runs, f"No training run found"
        return runs[0]

    def collect_run_df(self, run, metric, divisor, ema, date_unit):
        keys = [m for m in [metric, divisor, "session"] if m is not None]
        df = run.history(samples=self.samples, keys=keys)
        df["map_name"] = run.config["map"]["name"]
        if not df.empty:
            df.sort_values("_step", inplace=True)
            df["metric"] = df[metric]
            if divisor is not None:
                df["metric"] /= df[divisor]
            if ema:
                df["metric"] = df["metric"].ewm(span=ema, adjust=True).mean()
            if date_unit is not None:
                df["metric"] = pd.to_datetime(df["metric"], unit=date_unit)
        else:
            df["metric"] = np.nan
            df["_step"] = np.nan
            df["session"] = np.nan
        return df

    def collect_runs_df(self, runs, metric, divisor, ema, date_unit):
        if not runs:
            return pd.DataFrame(columns=["map_name", "metric", "_step", "session"])
        runs_df = [
            self.collect_run_df(run, metric, divisor, ema, date_unit) for run in runs
        ]
        return pd.concat(runs_df)

    def color_map(self, runs, env_color):
        return {
            run.config["map"]["name"]: env_color[run.config["name"]]
            if run.config["name"] in env_color
            else px.colors.qualitative.Vivid[i % len(px.colors.qualitative.Vivid)]
            for i, run in enumerate(runs)
        }

    def collect_figure(
        self,
        metric,
        title,
        fig_func=px.line,
        divisor=None,
        ema=0,
        date_unit=None,
        tickformat=None,
        env_color={},
    ):
        runs = self.collect_runs()
        df = self.collect_runs_df(runs, metric, divisor, ema, date_unit)
        fig = fig_func(
            df,
            color="map_name",
            x="_step",
            y="metric",
            template="plotly_dark",
            labels={"_step": "Round #", "map_name": "Maps"},
            color_discrete_map=self.color_map(runs, env_color),
        )
        fig.update_layout(
            title=dict(text=title, x=0.5, y=0.98),
            margin=dict(pad=10, t=94),
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
            font=self.font,
            legend=dict(
                title=None,
                font_size=16,
                orientation="h",
                yanchor="bottom",
                y=1.01,
                xanchor="center",
                x=0.5,
            ),
            height=480,
            width=760,
        )
        fig.update_yaxes(title=None, tickformat=tickformat)
        fig.update_traces(connectgaps=True)
        return fig

    def training_run_df(self, metric, ema):
        df = self.training_run().history(
            samples=self.samples, keys=[metric], x_axis="_runtime"
        )
        df["metric"] = df[metric]
        if not df.empty:
            df.sort_values("_runtime", inplace=True)
            df["_runtime"] /= 60 * 60  # s to hours
            if ema:
                df["metric"] = df["metric"].ewm(span=ema, adjust=True).mean()
        return df

    def training_figure(self, metric, title, ema=0, tickformat=None):
        fig = px.line(
            self.training_run_df(metric, ema),
            x="_runtime",
            y="metric",
            template="plotly_dark",
            labels={"_runtime": "Time (hours)"},
            color_discrete_sequence=px.colors.qualitative.Dark2,
        )
        fig.update_layout(
            title_text=title,
            title_x=0.5,
            margin=dict(pad=10),
            paper_bgcolor="rgba(0,0,0,0)",
            plot_bgcolor="rgba(0,0,0,0)",
            showlegend=False,
            font=self.font,
        )
        fig.update_yaxes(title=None, tickformat=tickformat)
        fig.update_traces(connectgaps=True)
        return fig

In [ ]:
# |exporti
class RunStatFactory(dict):
    @staticmethod
    def parse_run(run):
        df = pd.DataFrame(run.scan_history(page_size=10**9))
        medal_data = dict(
            best_medal="unfinished",
            unfinished=True,
            no_medal=False,
            bronze=False,
            silver=False,
            gold=False,
            author=False,
        )
        if "finish_racetime" in df and df["finish_racetime"].notnull().any():
            medal_data["best_medal"] = "no_medal"
            medal_data["unfinished"] = False
            medal_data["no_medal"] = True
            best_time = df["finish_racetime"].min()
            best_medal_time = None
            for medal, medal_time in run.config["map"]["medals"].items():
                better = best_time <= medal_time
                medal_data[medal] = better
                if better and (best_medal_time is None or medal_time < best_medal_time):
                    best_medal_time = medal_time
                    medal_data["best_medal"] = medal

        return dict(
            runtime=run.summary["_runtime"],
            map_name=run.config["map"]["sanitized_name"],
            **medal_data,
        )

    def run_stats(self, run):
        if run.id not in self or run.summary["_runtime"] != self[run.id]["runtime"]:
            self[run.id] = self.parse_run(run)
        return self[run.id]

    def runs_stats(self, runs):
        data = {run.id: self.run_stats(run) for run in runs}
        return pd.DataFrame.from_dict(data, orient="index")

## Layout

In [ ]:
# | exporti
def training_duration():
    text = html.P(
        [
            "Training time",
            html.Br(),
            html.Span(
                "0s",
                id="training-duration",
                style={"color": px.colors.qualitative.Dark2[0]},
            ),
        ]
    )
    children = [
        text,
        dcc.Interval(
            id="interval-duration",
            interval=1000,
            n_intervals=0,
        ),
        dcc.Store(data=0, id="training-duration-offset"),
    ]
    return html.Div(children)


def medals_obtained(df):
    if not len(df):
        return html.Div(
            [
                html.P("Medals obtained"),
                dbc.Spinner(color=px.colors.qualitative.Dark2[0]),
            ]
        )
    n_maps = len(df)
    counts = df["best_medal"].value_counts().reindex(medal_index, fill_value=0)
    text = html.P(
        [
            html.Span("Medals obtained"),
            html.Br(),
            html.Span(f"({n_maps} maps)", style={"font-size": 20}),
        ],
        className="pb-0 mb-0",
    )
    rows = [
        table_row(medal_icon(medal), f"{counts[medal]} / {n_maps}")
        for medal in medal_index
    ]
    medal_table = dbc.Table(
        rows,
        borderless=True,
        style={
            "color": px.colors.qualitative.Dark2[0],
            "width": "1%",
            "white-space": "nowrap",
        },
    )
    medal_table = dbc.Container(
        html.Div(medal_table),
        class_name="d-flex justify-content-center",
        fluid=True,
    )
    return html.Div([text, medal_table])


def graph(figure):
    return dcc.Graph(
        figure=figure,
        config={"displayModeBar": False},
        className="p-4",
    )


def layout(size, interval, border=False):
    switcher = dcc.Loading(
        dbc.Container(id="switcher", fluid=True),
        id="loading",
        type="dot",
    )
    switcher = dbc.Col(switcher, width=9, class_name="g-0 p-0 m-0")
    training_stats = dbc.Col(
        [
            training_duration(),
            html.Div(medals_obtained([]), id="medals-obtained"),
        ],
        style={"font-size": 24},
        class_name="g-0 p-0 m-0 text-center pt-4",
    )
    content = dbc.Row([switcher, training_stats], class_name="g-0 p-0 m-0")
    children = [
        html.Meta(httpEquiv="refresh", content=12 * 3600),
        dcc.Location(id="url", refresh=False),
        content,
        dcc.Interval(
            id="interval-component",
            interval=interval,
            n_intervals=0,
        ),
    ]
    return dbc.Container(
        children,
        class_name="position-relative p-0" + (" border border-light" if border else ""),
        style=size | {"background-color": "rgb(28,35,50)"},
        fluid=True,
    )

In [ ]:
# | export
def seconds2str(seconds):
    minutes, seconds = divmod(int(seconds), 60)
    hours, minutes = divmod(minutes, 60)
    days, hours = divmod(hours, 24)
    pairs = zip([days, hours, minutes, seconds], ["d", "h", "m", "s"])
    return " ".join(f"{value}{label}" for value, label in pairs if value)


class Dashboard(Thread):
    def __init__(
        self,
        entity=os.getenv("TMLEARN_WANDB_ENTITY", "trackmania-ai"),
        project=os.getenv("TMLEARN_WANDB_PROJECT", "tmaifoo"),
        size=dict(width=960, height=540),
        interval=15 * 1000,  # in milliseconds
        ema=50,
        debug=False,
        border=False,
        port=8050,
    ):
        super().__init__()
        self.debug = debug
        self.port = port
        self.ff = WandbFigureFactory(entity, project)
        self.rsf = RunStatFactory()
        self.env_color = dict()
        self.app = Dash(
            external_stylesheets=[dbc.themes.DARKLY, dbc.icons.FONT_AWESOME],
        )
        self.app.enable_dev_tools(dev_tools_silence_routes_logging=None)
        self.app.layout = layout(size, interval, border)

        self.collect_figures = [
            dict(
                metric="progress",
                title=f"Progress - ema {ema}",
                ema=ema,
                tickformat=".0%",
            ),
            dict(
                metric="total_reward",
                divisor="total_step_count",
                title=f"Reward per step - ema {ema}",
                ema=ema,
            ),
            dict(
                metric="no_respawn_reward",
                divisor="no_respawn_step_count",
                title=f"Reward per step - no respawn - ema {ema}",
                ema=ema,
            ),
            dict(
                metric="finish_racetime",
                title="Finish time",
                date_unit="ms",
                tickformat="%M:%S.%L",
                fig_func=px.scatter,
            ),
            dict(
                metric="finish_racetime",
                title=f"Finish time - ema {ema}",
                ema=ema,
                date_unit="ms",
                tickformat="%M:%S.%L",
            ),
            dict(
                metric="no_respawn_finish_racetime",
                title=f"Finish time - no respawn - ema {ema}",
                ema=ema,
                date_unit="ms",
                tickformat="%M:%S.%L",
            ),
        ]
        self.training_figures = [
            dict(
                metric="mean_errors",
                title="Training error in reward estimation - smoothed",
                ema=1000,
            )
        ]

        @self.app.callback(Output("url", "refresh"), Input("url", "href"))
        def websockets(href):
            self.env_color = dict(furl(href).args)
            return False

        @self.app.callback(
            Output("switcher", "children"),
            Input("interval-component", "n_intervals"),
        )
        def update_line_chart(n):
            n_collect = len(self.collect_figures)
            n_training = len(self.training_figures)
            n = n % (n_collect + n_training)
            if n < n_collect:
                fig = self.ff.collect_figure(
                    **self.collect_figures[n],
                    env_color=self.env_color,
                )
                return graph(fig)
            n -= n_collect
            fig = self.ff.training_figure(**self.training_figures[n])
            return graph(fig)

        @self.app.callback(
            Output("training-duration", "children"),
            Output("training-duration-offset", "data"),
            Input("interval-duration", "n_intervals"),
            Input("training-duration-offset", "data"),
        )
        def update_training_duration(n, offset):
            if not offset:
                try:
                    offset = int(time() - self.ff.training_run().summary["_runtime"])
                except AssertionError:
                    offset = 0
            return [seconds2str(time() - offset), offset]

        @self.app.callback(
            Output("medals-obtained", "children"),
            Input("interval-component", "n_intervals"),
        )
        def update_medals_obtained(n):
            group = self.ff.training_run().group
            runs = self.ff.collect_runs(state=None, group=group)
            df = self.rsf.runs_stats(runs)
            return medals_obtained(df)

        self.start()

    def run(self):
        self.app.run_server(debug=self.debug, port=self.port)

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()